In [ ]:
%matplotlib inline
import numpy as np
import math
import sys
import glob
import re
import random
import collections
import os
import sys
import pandas as pd
import deepcut
from pathlib import Path
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense ,GRU ,TimeDistributed ,LSTM ,Bidirectional,Flatten ,Dropout
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import Adam
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib as mpl
import requests
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt, mpld3
import seaborn as sns
from scipy.stats import norm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# https://www.kaggle.com/itoeiji/deep-reinforcement-learning-on-stock-data
# https://github.com/rlcode/reinforcement-learning/blob/master/2-cartpole/1-dqn/cartpole_dqn.py

จากหลาย Document การใช้ Dense จะใช้ relu เป็น activation และ layer  สุดท้ายให้ใช้ Dense โดยไม่ต้องใส่ Activation
https://markelsanz14.medium.com/introduction-to-reinforcement-learning-part-3-q-learning-with-neural-networks-algorithm-dqn-1e22ee928ecd
https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c

In [ ]:
def timestamp2str(x):
    return datetime.utcfromtimestamp(x).strftime('%Y-%m-%d')

In [ ]:
df = pd.read_csv('/Users/chakritphain/Downloads/TFEX_S50U2020_1_19-10_to 04-12.csv')
dftmp = df[['open','high','low','close','Volume']]
dftmp['Volume'] = dftmp.Volume.apply(lambda x: x.replace(",",""))
#dftmp

In [ ]:
dftmp.Volume  = dftmp.Volume.astype('float64')

In [ ]:
dftmp.head(10)

In [ ]:
def Value2Diff(series):
    prev = series[:-1]
    now = series[1:]
    return np.concatenate(([0],now-prev))
#def DiffClose()
def ma(series,num):
    tmp = series[num:].values
    for i in range(1,num):
        #print(i)
        tmp = series[num-i:-i].values + tmp 
    tmp = tmp/num
    avg = np.average(series)
    tmp = np.concatenate((series[:num],tmp))
    return tmp

In [ ]:
dftmp['vma5'] = ma(dftmp.Volume,5)
dftmp['vma15'] = ma(dftmp.Volume,15)
dftmp['vma30'] = ma(dftmp.Volume,30)
dftmp['cma5'] = ma(dftmp.close,5)
dftmp['cma15'] = ma(dftmp.close,15)
dftmp['cma30'] = ma(dftmp.close,30)

In [ ]:
#pd.DataFrame(ma(dftmp.Volume,5))
#dftmp.Volume

In [ ]:
o = Value2Diff(dftmp.open.values)
#o = dftmp.open - dftmp.close  #เปิด-ปิด
#h = Value2Diff(dftmp.high.values)
h = dftmp.high - dftmp.close #สูง-ปิด
#l = Value2Diff(dftmp.low.values)
l = dftmp.close - dftmp.low #ปิด-ต่ำ
c = Value2Diff(dftmp.close.values) #ปิดตอนนี้-ปิดเมื่อกี้
cd5 = dftmp.close - dftmp.cma5
c5 = Value2Diff(dftmp.cma5.values)
cd15 = dftmp.close - dftmp.cma15
c15 = Value2Diff(dftmp.cma15.values)
cd30 = dftmp.close - dftmp.cma30
c30 = Value2Diff(dftmp.cma30.values)
c5d15 = dftmp.cma5 - dftmp.cma15
c5d30 = dftmp.cma5 - dftmp.cma30
v = Value2Diff(dftmp.Volume.values) #ปิดตอนนี้-ปิดเมื่อกี้
vd5 = dftmp.Volume - dftmp.vma5
v5 = Value2Diff(dftmp.vma5.values)
vd15  = dftmp.Volume - dftmp.vma15
v15 = Value2Diff(dftmp.vma15.values)
vd30  = dftmp.Volume - dftmp.vma30
v30 = Value2Diff(dftmp.vma30.values)
v5d15 = dftmp.vma5 - dftmp.vma15
v5d30 = dftmp.vma5 - dftmp.vma30
df_diff = pd.DataFrame({'o':o,'h':h,'l':l,'c':c,'v':v,'v5':v5,'v15':v15,'v30':v30,'vd5':vd5,'vd15':vd15,'vd30':vd30,'v5d15':v5d15,'v5d30':v5d30,'c5':c5,'c15':c15,'c30':c30,'cd5':cd5,'cd15':cd15,'cd30':cd30,'c5d15':c5d15,'c5d30':c5d30})
display(df_diff.head(10))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
scaler.fit(df_diff) #Make Model MinMaxScaler
df_diff_norm = pd.DataFrame(scaler.transform(df_diff),columns = df_diff.columns)
df_diff_norm

In [ ]:
scaler.inverse_transform(np.array([[0,0,0,0.282132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))[0,3]

In [ ]:
scaler.inverse_transform(df_diff_norm.iloc[0:20]).T[3]

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(251).hist(df_diff['o'])
plt.subplot(252).hist(df_diff['h'])
plt.subplot(253).hist(df_diff['l'])
plt.subplot(254).hist(df_diff['c'])
plt.subplot(255).hist(df_diff['v'])
plt.subplot(256).hist(df_diff_norm['o'])
plt.subplot(257).hist(df_diff_norm['h'])
plt.subplot(258).hist(df_diff_norm['l'])
plt.subplot(259).hist(df_diff_norm['c'])
plt.subplot(2,5,10).hist(df_diff_norm['v'])
plt.subplot(251).legend(['o'])
plt.subplot(252).legend(['h'])
plt.subplot(253).legend(['l'])
plt.subplot(254).legend(['c'])
plt.subplot(255).legend(['v'])

In [ ]:
#For LSTM
import tensorflow as tf
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    #dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    #dataset = dataset.batch(batch_size).prefetch(1)
    return dataset


#for element in dataset:
#    print(element)

In [ ]:
window_size = 5
batch_size = 12
shuffle_buffer_size = 10

#data_series = np.array(list(windowed_dataset(df_diff_norm.iloc[:].values,window_size,batch_size,shuffle_buffer_size)))
#data_series.shape

In [ ]:
def westerncandlestick(ax, Data, width=0.2, colorup='k', colordown='r', linewidth=0.5):

    Open, Hight, Low,Close, Volumn, v5 ,v15 ,v30,c5,c15,c30 = Data
    Date = np.array(range(len(Open)))
    Date  = Date.astype('float32')
    Color = ['b']
    for i in range(1,len(Date)): 
        Color.append(['r','k'][Open[i-1]<Close[i]])
    OFFSET = .4

    #Hight Low Lines
    ax.vlines(Date,Low,Hight,Color)

    #Open Lines
    ax.hlines(Open,Date-OFFSET,Date,Color)
    #Close Lines
    ax.hlines(Close,Date,Date+OFFSET,Color)

    ax.autoscale_view()

    return

In [ ]:
df_diff_norm.iloc[:100].T

In [ ]:
fromlim = -100
lim = -1
f,ax = plt.subplots(figsize=(20,10))
ax.plot(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.iloc[fromlim:lim].cma5,'r-')
ax.plot(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.iloc[fromlim:lim].cma15,'g-')
ax.plot(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.iloc[fromlim:lim].cma30,'b-')
#f.subplots_adjust(right=0.75)
vol = ax.twinx()
fromlim = -100
lim = -1
vol.bar(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.Volume.values[fromlim:lim])
vol.plot(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.iloc[fromlim:lim].vma5,'r-')
vol.plot(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.iloc[fromlim:lim].vma15,'g-')
vol.plot(list(range(len(dftmp.iloc[fromlim:lim]))),dftmp.iloc[fromlim:lim].vma30,'b-')
westerncandlestick(ax,dftmp.iloc[fromlim:lim].T.values)

#host.set_xlim(0, 210)
ax.set_ylim(dftmp.iloc[fromlim:lim].low.min()-10, dftmp.iloc[fromlim:lim].high.max())
vol.set_ylim(0, dftmp.iloc[fromlim:lim].Volume.max()*2)


In [ ]:
import time
import copy
import numpy as np
import pandas as pd
import numpy as np # linear algebra
#init_notebook_mode()
#from collections import deque

In [ ]:
class Environment1:
    def __init__(self, data,scaler):
        self.data = data
        #self.history_t = history_t
        self.scaler = scaler
        self.window_size = window_size+1
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        #self.profits = 0
        self.positions = []
        self.position_value = 0
        self.buyed = 0 #0 = ยังไม่ซื้อ , 1 = L , 2 = S
        #self.rewardSat = 0 มันคืออันเดียวกับ self.position_value เลือกอันใดอันหนึ่ง (มูลค่าสะสม)
        #self.history = [0 for _ in range(self.history_t)]
        #print(self.addstate(self.data[0],self.buyed,self.position_value))
        #return [self.position_value] + self.history # obs
        #print(self.data.iloc[0])
        return self.addstate(self.data.iloc[0],self.buyed,self.position_value)
    def plot(self,ax):
        r = self.scaler.inverse_transform(self.data).T[3]
        ax.plot(list(range(len(r))),r,'y+')
        return
    
    def addstate(self,data_stage,buyed,position_value):
        if buyed == 0:
            state = np.append(data_stage.T,1) #
            state = np.append(state.T,0) #
            state = np.append(state.T,0) #
        elif buyed == 1:
            state = np.append(data_stage.T,0) #
            state = np.append(state.T,1) #
            state = np.append(state.T,0) #
        elif buyed == 2:
            state = np.append(data_stage.T,0) #
            state = np.append(state.T,0) #
            state = np.append(state.T,1) #            
            
        pv = self.ScalerPV(position_value)
        
        state = np.append(state,pv)
        #print(state)
        return state.T
    #def buyedTransform(self,buyed):
    #    return buyed/2
    def ScalerPV(self,x):
        return (x + 5) / 10
    
    def step(self, act):
        # act = 0: stay, 1: buy, 2: sell
        if act == 0:
            reward = 0
            if self.buyed == 1:
                r = self.scaler.inverse_transform(np.array([[0,0,0,self.data.iloc[self.t]['c'],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))[0,3]
                self.positions.append(r)
                #self.position_value = self.position_value + r
            elif  self.buyed == 2:
                r = self.scaler.inverse_transform(np.array([[0,0,0,self.data.iloc[self.t]['c'],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))[0,3]
                self.positions.append(r) 
                #self.position_value = self.position_value - r
        elif act == 1: #buy L
            if self.buyed == 1 or self.buyed == 2:
                reward = -1 #ถ้ามีการซื้อแล้ว และมีการซื้อซ้ำ ให้ลบ 1 (ต้องขายก่อนจึงจะซืื้อใหม่)
                r = self.scaler.inverse_transform(np.array([[0,0,0,self.data.iloc[self.t]['c'],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))[0,3]
                self.positions.append(r) #แต่ก็ยัง append state ปัจจุบันอยู่ดี
            else:
                #r = self.scaler.inverse_transform(np.array([[0,0,0,self.data.iloc[self.t]['c'],0]]))[0,3]
                self.positions.append(0)
                self.buyed = 1
                reward = 0.01 #prefer buy more than stay #-0.3
        elif act == 3: #buy S
            if self.buyed == 1 or self.buyed == 2:
                reward = -1 #ถ้ามีการซื้อแล้ว และมีการซื้อซ้ำ ให้ลบ 1 (ต้องขายก่อนจึงจะซืื้อใหม่)
                r = self.scaler.inverse_transform(np.array([[0,0,0,self.data.iloc[self.t]['c'],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]))[0,3]
                self.positions.append(r) #แต่ก็ยัง append state ปัจจุบันอยู่ดี
            else:
                #r = self.scaler.inverse_transform(np.array([[0,0,0,self.data.iloc[self.t]['c'],0]]))[0,3]
                self.positions.append(0)
                self.buyed = 2
                reward = 0.01 #prefer buy more than stay #-0.3        
        elif act == 2: # sell ตอนขายจึงจะได้ Reward
            if self.buyed == 0:
                reward = -1 #ถ้ายังไม่เคยซื้อแล้วกดขายให้ลบ 1 (ต้องซืื้อก่อนจึงจะขายได้)
            elif self.buyed == 1:#L
                profits = 0
                for p in self.positions:
                    profits += p
                reward = profits - 0.3
                self.positions = []
                self.position_value = 0
                self.buyed = 0                
            elif self.buyed == 2:#S
                profits = 0
                for p in self.positions:
                    profits += p
                reward = -profits - 0.3
                self.positions = []
                self.position_value = 0
                self.buyed = 0   
        
        # set next time
        #print(self.t,len(self.data))
        if self.t >= len(self.data)-1:
            self.done = True
            state = self.addstate(self.data.iloc[self.t, :],self.buyed,self.position_value)
            return state ,reward, self.done

        self.t += 1
        self.position_value = 0
        for p in self.positions:
            if p > 0:
                stay_rate  = 0.001
            else:
                stay_rate  = -0.001
            self.position_value = self.position_value + p + stay_rate
            
        if self.buyed == 1:
            self.position_value = self.position_value
        elif self.buyed == 2:
            self.position_value = -self.position_value    
            
        if self.position_value < -5: #บังคับขาย ถ้าขาดทุนเกิน 1  บาทให้ขายทันที (ไม่อยากให้ self.position_value ต่ำกว่า 1 บาทด้วย)
            #เพราะใช้ Relu ค่าเลยติดลบไม่ได้
            reward += -6
        
        #จำนวน Array ของ self.positions คือค่า Stay ยิ่งนานก็ให้ rewardเยอะ
        #เนื่องจาก Deep Learning ต้องการแค่เลข 0-1 จึงสามารถกำหนดได้เพียง + และ - โดย หากติดลบให้ 0 หากเป็น + ให้ 1
        #self.position_value = 0 if self.position_value < 0 else 1
        #self.history.pop(0)
        #self.history.append(self.data.iloc[self.t]['c'] - self.data.iloc[(self.t-1)]['c'])
        #fc = self.data.iloc[self.t, :].c
        #fo = self.data.iloc[self.t, :].o
        #fl = self.data.iloc[self.t, :].l
        #fh = self.data.iloc[self.t, :].h
        #fv = self.data.iloc[self.t, :].v 
        #return [self.position_value] + self.history, reward, self.done # obs, reward, done
        #จริงๆ ควรเอา self.position_value เป็น state ด้วย
        #แต่ติดที่ค่ามันมากกว่า 1 เข้า deep learning แล้วอาจมีปัญหาเพราะยังไม่ได้ Normalize
        #จึงเอาเท่านี้ก่อน
        #return [fo,fh,fl,fc,fv,self.buyed,self.position_value], reward, self.done
        return self.addstate(self.data.iloc[self.t, :],self.buyed,self.position_value) ,reward, self.done

In [ ]:
env = Environment1(df_diff_norm,scaler)

In [ ]:
env = Environment1(df_diff_norm,scaler)
act = ['stay','buy','sell']
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print(act[pact])
    print(env.step(pact))
env.plot(plt)

In [ ]:
state_size = 25
action_size = 4

In [ ]:
learning_rate = 0.01
#del(model)
model = Sequential()
model.add(Dense(40, input_dim=state_size, activation='sigmoid', kernel_initializer='he_uniform'))
#model.add(Dropout(0.2))
model.add(Dense(4, input_dim=state_size, activation='sigmoid', kernel_initializer='he_uniform'))
model.add(Dense(action_size, activation='linear',kernel_initializer='he_uniform'))
model.summary()
model.compile(loss='mse', optimizer=Adam(lr=learning_rate))

In [ ]:
import  datetime
def append_sample(state, action, reward, next_state, done,epsilon_min,epsilon_decay,epsilon,memory):
    memory.append((state, action, reward, next_state, done))
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
    return [epsilon,memory]

def train_model(model,memory,train_start,batch_size,state_size,discount_factor):
    if len(memory) < train_start:
        return
    batch_size = min(batch_size, len(memory)) #ค่าไหนน้อยกว่ากันเอาค่านั้นระหว่าง batch_size  และ จำนวน data ใน memory
    mini_batch = random.sample(memory, batch_size) # สุ่มจาก memory มาเท่ากับจำนวน batch_size เพื่อเอาไป Train

    state = np.zeros((batch_size,state_size))
    next_state= np.zeros((batch_size, state_size))
    action, reward, done = [], [], []

    for i in range(batch_size):
        state[i] = mini_batch[i][0]
        action.append(mini_batch[i][1])
        reward.append(mini_batch[i][2])
        next_state[i] = mini_batch[i][3]
        done.append(mini_batch[i][4])

    state_Q = model.predict(state) #สร้างตาราง Q-Table ขึ้นมา

    next_state_Q = model.predict(next_state)

    for i in range(batch_size):
        # Q Learning: get maximum Q value at s' from target model
        if done[i]:
            state_Q[i][action[i]] = reward[i] #ใส่ Reward ที่ Q-Table
        else:
            #กุญแจของ Q Learning อยู่ตรงนี้
            state_Q[i][action[i]] = reward[i] + discount_factor * (np.amax(next_state_Q[i]))
    log_dir = "../data/logs/stock_ql_V4.0_25dim1Hidden/" + datetime.datetime.now().strftime("%Y%m%d")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # and do the model fit!
    filename = "./stock_ql_V4.0_25dim.f5"
    if os.path.isfile(filename+".index"):
        #print("Load ../dql_stock_ql_V3_weight.f5")
        a = model.load_weights(filename)
        #print(a)
    history = model.fit(state, state_Q, batch_size=batch_size,epochs=30, verbose=0,callbacks=[tensorboard_callback])
    model.save_weights(filename)
    return [model,history]

In [ ]:
%%time
fromlim = random.randint(-1000,-101)
lim = fromlim+100
env = Environment1(df_diff_norm.iloc[fromlim:lim],scaler)
# Hyperparameters
alpha = 0.8
gamma = 0.8
epsilon = 0.5
epsilon_decay = 0.99999
discount_factor = 0.9
epsilon_min = 0.01 #0.01
train_start = 10
batch_size  = 100
total_action = 12
results  = []
# For plotting metrics
all_epochs = []
all_penalties = []
memory = []
scores = []
episodes = []
mse_loss = []
fromlim = -822
lim = -722
for i in range(1,10000+1):
    fromlim = random.randint(-1000,-101)
    lim =  fromlim+100
    print('Random=',fromlim,lim,end=" ")
    state = np.zeros(state_size)
    score = 0
    arr_score = []
    arr_reward = []
    arr_position_v = []
    arr_state = []
    arr_action = []
    epochs, penalties, reward, s = 0, 0, 0 ,0
    #epsilon = 0.1
    done = False
    del(env)
    env = Environment1(df_diff_norm.iloc[fromlim:lim],scaler)
    env.reset()
    if i % 100  ==  0:
        plt.figure(figsize=(20,10))
        ax1 = plt.subplot(411)
        ax2 = plt.subplot(412)
        ax3 = plt.subplot(413)
        ax4 = plt.subplot(414)
        vol = ax1.twinx()
        ax1.plot(list(range(lim-fromlim)),dftmp.iloc[fromlim:lim]['close'])
        ax1.plot(list(range(lim-fromlim)),dftmp.iloc[fromlim:lim].cma5,'r-')
        ax1.plot(list(range(lim-fromlim)),dftmp.iloc[fromlim:lim].cma15,'g-')
        vol.bar(list(range(lim-fromlim)),dftmp.iloc[fromlim:lim].Volume)
        vol.plot(list(range(lim-fromlim)),dftmp.iloc[fromlim:lim].vma5,'r-')
        vol.plot(list(range(lim-fromlim)),dftmp.iloc[fromlim:lim].vma15,'g-')
        ax1.set_ylim(dftmp.iloc[fromlim:lim].low.min()-10, dftmp.iloc[fromlim:lim].high.max())
        vol.set_ylim(0, dftmp.iloc[fromlim:lim].Volume.max()*2)
    while not done:
        if np.random.rand() <= epsilon:
            action = random.randrange(action_size)
            arr_action_tmp = np.zeros(action_size)
            color = ['dont care','g+','r+','b+']
        else:
            color = ['dont care','go','ro','bo']
            #action = np.argmax(Qtable[state]) # Exploit learned values
            arr_action_tmp =  model.predict(np.array(state).reshape(1,-1))[0]
            action = np.argmax(arr_action_tmp)
        arr_action.append(arr_action_tmp)
        if action != 0:
            ax1.plot(s,dftmp.iloc[fromlim+s]['close'],color[action]) 
        s += 1 
        next_state, reward, done = env.step(action)
        #print(next_state)
        score += reward
        arr_reward.append(reward)
        arr_score.append(score)
        arr_position_v.append(next_state[-1])
        #state2 = np.reshape(state, [1, state_size])
        #next_state2 = np.reshape(next_state, [1, state_size]) #DQN
        epsilon,memory = append_sample(state, action, reward, next_state, done,epsilon_min,epsilon_decay,epsilon,memory) #DQN
        state = next_state
        arr_state.append(state)
        #old_value = Qtable[state,action]
        #next_max = np.max(Qtable[next_state])
        #new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        #Qtable[state,action] = new_value
        #print(model,memory,train_start,batch_size,state_size,discount_factor)
    if i % 100  ==  0:
        ax2.plot(arr_reward,'b+')
        ax2.plot(arr_score,'m-')
        ax2.plot(arr_position_v,'r.')
        ax2.axhline(y=0.0, color='g', linestyle='-')
        ax2.axhline(y=-5, color='b', linestyle='-')
        env.plot(ax2)
        ax2.legend(["reward (P&L)","arr_reward_cum (P&L)"])
        ax3.plot(arr_action)
        ax3.legend(['Stay','L','Sell','S'])
        ax4.plot(arr_state)
        ax4.legend(['o','h','l','c','v','v5','v15','v30','v5d15','v5d30','c5','c15','c30','c5d15','c5d30','stay','L','S','pValue'])

    print("Epoch=" +str(i) + ", Epsilon=%.2f"%epsilon + ",Score=%.2f"%score)
    plt.show(arr_state)
    scores.append(score)
    #print("Score=%.2f,%.2f" % (score , epsilon))
    result = train_model(model,memory,train_start,batch_size,state_size,discount_factor)
    if result:
        model,history = result
        mse_loss.append(history.history['loss'])
        #print('loss = ',history.history['loss'][0])
    if i % 10  ==  0:
        plt.figure(figsize=(20,5))
        plt.plot(mse_loss[:])
        plt.axhline(y=0.05, color='b', linestyle='-')
        plt.legend(["mse_loss"])
        plt.show()
        plt.figure(figsize=(20,5))
        plt.plot(scores[:])
        plt.axhline(y=0.0, color='b', linestyle='-')
        plt.legend(["Scores"])
        #plt.ylim(0,10)
        plt.show()